In [1]:
import json


In [2]:
import json

# 파일 경로
RAW_POI_JSON = "./rest_data.json"
OUT_JSON = "./rest_data_poi_id.json"

with open(RAW_POI_JSON, "r", encoding="utf-8") as f:
    pois = json.load(f)

for poi in pois:
    # id → poi_id로 변경
    if "id" in poi:
        poi["poi_id"] = poi.pop("id")

with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(pois, f, ensure_ascii=False, indent=2)

print(f"✅ id → poi_id 변환 완료: {OUT_JSON}")


✅ id → poi_id 변환 완료: ./rest_data_poi_id.json


In [ ]:
# 파일 경로
RAW_POI_JSON = "./rest_data_poi_id.json"          # 원본 POI JSON (list)
PLACE_JSON = "./poi_with_id_region.json"  # place 메타 JSON (list)
OUT_JSON = "embedding_restaurant.json"      # 결과 embedding용으로 사용


In [7]:
with open(RAW_POI_JSON, "r", encoding="utf-8") as f:
    raw_pois = json.load(f)

with open(PLACE_JSON, "r", encoding="utf-8") as f:
    places = json.load(f)

print(f"raw_pois: {len(raw_pois)}")
print(f"places: {len(places)}")


raw_pois: 649
places: 3350


In [8]:
place_lookup = {
    p["poi_id"]: {
        "place_id": p["place_id"],
        "category": p["category"],
        "region": p["region"],
    }
    for p in places
}


In [9]:
missed = 0

for poi in raw_pois:
    poi_id = poi.get("poi_id")

    if poi_id in place_lookup:
        poi.update(place_lookup[poi_id])
    else:
        missed += 1

print(f"❗ 매칭 안 된 POI 수: {missed}")


❗ 매칭 안 된 POI 수: 0


In [10]:
TOP_KEYS = ["place_id", "category", "region"]


In [11]:
def reorder_poi_fields(poi: dict) -> dict:
    ordered = {}

    # 1) 맨 위로 올릴 키들
    for key in TOP_KEYS:
        if key in poi:
            ordered[key] = poi[key]

    # 2) 나머지 키들 (기존 순서 유지)
    for key, value in poi.items():
        if key not in ordered:
            ordered[key] = value

    return ordered


In [12]:
pois_reordered = [reorder_poi_fields(poi) for poi in raw_pois]


In [13]:
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(pois_reordered, f, ensure_ascii=False, indent=2)

print(f"✅ Saved merged POI JSON → {OUT_JSON}")


✅ Saved merged POI JSON → embedding_cafe.json
